In [93]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

# Replace 'your_image.mha' with the actual file path
image_path = 'mia-result/2024-10-17-09-57-49/117122_SEG.mha'
image = sitk.ReadImage(image_path)

# Convert the image to a NumPy array if needed
image_array = sitk.GetArrayFromImage(image)

# Print the image dimensions
print(f"Image size: {image.GetSize()}")
print(f"Image array shape: {image_array.shape}")

Image size: (197, 233, 189)
Image array shape: (189, 233, 197)


In [92]:
# Perform binary erosion
# eroded_image = sitk.BinaryErode(image, kernelRadius=[0, 1, 0], foregroundValue=1)
# eroded_image_array = sitk.GetArrayFromImage(eroded_image)

erode_filter = sitk.BinaryErodeImageFilter() 
erode_filter.SetForegroundValue(1) 
erode_filter.SetKernelType(sitk.sitkBall) 
eroded_image = erode_filter.Execute(image)
eroded_image_array = sitk.GetArrayFromImage(eroded_image)

# Fill holes in the eroded image
# filled_image = sitk.BinaryFillhole(eroded_image)
fillhole_filter = sitk.BinaryFillholeImageFilter() 
fillhole_filter.SetFullyConnected(True) 
fillhole_filter.SetForegroundValue(1) 
filled_image = fillhole_filter.Execute(eroded_image) 
filled_image_array = sitk.GetArrayFromImage(filled_image)

# Perform binary dilation on the filled image
# dilated_image = sitk.BinaryDilate(filled_image, kernelRadius=[1, 1, 1])
dilate_filter = sitk.BinaryDilateImageFilter() 
dilate_filter.SetForegroundValue(1) 
dilate_filter.SetKernelType(sitk.sitkBall)
dilated_image = dilate_filter.Execute(filled_image) 
dilated_image_array = sitk.GetArrayFromImage(dilated_image)

# TRY TO USE CONNECTED COMPONENTS ON EACH INDIVUAL LABEL OF THE MASK AND AFTER COMBINE AGAIN

# Determine the slices to visualize (if the image is 3D)
if len(image_array.shape) == 3:
    slice_indices = [image_array.shape[0] // 2 - 10, 
                     image_array.shape[0] // 2,  # 2nd slice (middle)
                     image_array.shape[0] // 2 + 10]
    
    # Prepare the plots
    original_slices = [image_array[i, :, :] for i in slice_indices]
    eroded_slices = [eroded_image_array[i, :, :] for i in slice_indices]
    filled_slices = [filled_image_array[i, :, :] for i in slice_indices]
    dilated_slices = [dilated_image_array[i, :, :] for i in slice_indices]
else:
    original_slices = [image_array]
    eroded_slices = [eroded_image_array]
    filled_slices = [filled_image_array]
    dilated_slices = [dilated_image_array]

# Create a colormap for visualization
cmap = plt.colormaps.get_cmap('tab20')

# Set up the plot
fig, axes = plt.subplots(len(slice_indices), 4, figsize=(20, 15))

# Loop through the slices and plot
for i, idx in enumerate(slice_indices):
    # Original image
    axes[i, 0].imshow(original_slices[i], cmap=cmap)
    axes[i, 0].set_title(f'Original Image (Slice {idx})')
    axes[i, 0].axis('off')

    # Eroded image
    axes[i, 1].imshow(eroded_slices[i], cmap=cmap)
    axes[i, 1].set_title(f'Eroded Image (Slice {idx})')
    axes[i, 1].axis('off')

    # Filled image
    axes[i, 2].imshow(filled_slices[i], cmap=cmap)
    axes[i, 2].set_title(f'Filled Image (Slice {idx})')
    axes[i, 2].axis('off')

    # Dilated image
    axes[i, 3].imshow(dilated_slices[i], cmap=cmap)
    axes[i, 3].set_title(f'Dilated Image (Slice {idx})')
    axes[i, 3].axis('off')

# Show the plot in a pop-up window
plt.tight_layout()
plt.show()
